In [1]:
base64abc = [
    "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M",
    "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z",
    "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m",
    "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z",
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "+", "/"
];

base64codes = [
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
    255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 62, 255, 255, 255, 63,
    52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 255, 255, 255, 0, 255, 255,
    255, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
    15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 255, 255, 255, 255, 255,
    255, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
    41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51
];

In [2]:
def bytesToBase64(bytes): 
    result = ''
    i = 2
    l = len(bytes)
    for i in range(2,l,3):
        result += base64abc[bytes[i - 2] >> 2];
        result += base64abc[((bytes[i - 2] & 0x03) << 4) | (bytes[i - 1] >> 4)];
        result += base64abc[((bytes[i - 1] & 0x0F) << 2) | (bytes[i] >> 6)];
        result += base64abc[bytes[i] & 0x3F];
    i += 3
    if (i == l + 1): 
        result += base64abc[bytes[i - 2] >> 2];
        result += base64abc[(bytes[i - 2] & 0x03) << 4];
        result += "==";
    if (i == l): 
        result += base64abc[bytes[i - 2] >> 2];
        result += base64abc[((bytes[i - 2] & 0x03) << 4) | (bytes[i - 1] >> 4)];
        result += base64abc[(bytes[i - 1] & 0x0F) << 2];
        result += "=";
    return result


def base64ToBytes(string): 
    if (len(string) % 4 != 0): 
        print("Unable to parse base64 string.")
        return 
    index = string.index("=")
    if (index != -1 and index < len(string) - 2):
        print("Unable to parse base64 string.")
        return
    missingOctets = 0
    if string[-2:] == "==":
        missingOctets = 2
    elif string[-1] == "=":
        missingOctets = 1
    else:
        missingOctets = 0
    n = len(string)
    result = [0 for x in range(3 * (n / 4))]
    buffer = None
    i = 0 
    j = 0
    while i < n:
        buffer = base64codes[ord(string[i])] << 18 | base64codes[ord(string[i+1])] << 12 | base64codes[ord(string[i+2])] << 6 | base64codes[ord(string[i+3])];
        result[j] = buffer >> 16;
        result[j + 1] = (buffer >> 8) & 0xFF;
        result[j + 2] = buffer & 0xFF;
        i += 4 
        j += 3
    return result[: len(result)-missingOctets];

In [3]:
def hex2bytes(string):
    normal = "0"+string if len(string) % 2 == 1 else string # Make even length
    bytes_ = [0]*(len(normal) // 2);
    for index in range(len(bytes_)): 
        c1 = ord(normal[index*2]) 
        c2 = ord(normal[index*2+1])
        n1 = c1 - (48 if c1 < 58 else 87) 
        n2 = c2 - (48 if c2 < 58 else 87) 
        bytes_[index] = n1 * 16 + n2
    return bytes_

def bytes2hex(b):
    chars = [0 for i in range(len(b) * 2)]
    alpha = ord('a') - 10
    digit = ord('0')

    p = 0;
    for i in range(len(b)):
        nibble = rshift(b[i], 4)
        chars[p] = nibble + alpha if nibble > 9 else nibble + digit
        p+=1
        nibble = b[i] & 0xF;
        chars[p] = nibble + alpha if nibble > 9 else nibble + digit
        p+=1
    return "".join([chr(x) for x in chars])

def rshift(val, n): return (val % (1 << 64)) >> n

In [4]:
def int_to_bytes(num):
    hex_ = hex(num).lstrip("0x").rstrip("L").zfill(32*2)
    return hex2bytes(hex_)[::-1]

def bytes_to_int(b):
    hex_ = bytes2hex(b[::-1])
    y = hex_[:len(hex_)/2]
    x = hex_[len(hex_)/2:]
    return int("0x"+x, 0), int("0x"+y, 0)


In [5]:
def pack(x, y):
    x_bytes = int_to_bytes(x)
    y_bytes = int_to_bytes(y)
    concat = bytesToBase64(x_bytes+y_bytes)
    return concat

def unpack(b64):
    receive_bytes = base64ToBytes(b64)
    x, y = bytes_to_int(receive_bytes)
    return x, y

In [6]:
# (u, v) = ((1+y)/(1-y), sqrt(-486664)*u/x)
def twisted_to_montg(p, x, y, d=-486664):
    gf=GF(p)
    X_P = gf(x)
    Y_P = gf(y)
    u = (1+Y_P)/(1-Y_P) 
    v = gf(d).sqrt()*u/X_P
    return (u, v)

# (x, y) = (sqrt(-486664)*u/v, (u-1)/(u+1))
def montg_to_twisted(p, u, v, d=-486664):
    gf=GF(p)
    x = gf(d).sqrt()*u/v
    y = (u-1)/(u+1)
    X_P = gf(x)
    Y_P = gf(y)
    return X_P, Y_P

In [7]:
def create_point(p=2^255-19, a1=0, a2=486662, a3=0, a4=1, a6=0):
    E = EllipticCurve(GF(p), [a1, a2, a3, a4, a6])
    print(E)

    order = E.order()
    factors = prime_factors(order)

    valid = []
    for factor in factors:
        if factor <= 2**35:
            valid.append(factor)
    if len(valid) == 0:
        return None, None, None
    
    prime = valid[-1]
    G = E.gen(0) * int(order / prime)  # G.order = prime
    print("G.order =", G.order(), "; prime =", prime)
    
    return prime, E, G

In [8]:
import requests

# input: point on montgomery curve.
# output: point received after multiplication by prism value on an ork, converted to montgomery form.
def send_point(G, url, d=-486664):
    G_twist = montg_to_twisted(p, G[0], G[1], d)
    package = pack(G_twist[0], G_twist[1])

    # send package to the appropriate url
    payload={'point': package}
    response = requests.request("POST", url, data=payload)
    receive = response.text
    P_twist = unpack(receive)
    
    P = twisted_to_montg(p, P_twist[0], P_twist[1], d)
    return P

In [9]:
def compute_discrete_log(E, G, P):
    try:
        Q = E(P[0], P[1])
    except:
        print("point doesn't exist.")
        return None
    try:
        log = G.discrete_log(Q)
        print("[+] discrete log:", log)
        return log
    except:
        print("[-] discrete log calculation failed.")
        return None

In [ ]:
import time

p = 2^255-19
dlogs = []
primes = []
for i in range(42):
    print("------", i+1, "------")
    b = randint(1, 4444444)
    d = -b - 2
    prime, E, G = create_point(a2=b)
    if prime == None:
        continue
    
    try:
#         P = send_point(G.xy(), "http://localhost:7001/Apply/Prism", d=d)
        P = send_point(G.xy(), "https://ork2.westus2.cloudapp.azure.com/Apply/Prism", d=d)
    except:
        continue
    log = compute_discrete_log(E, G, P)
    if log != None:
        dlogs.append(log)
        primes.append(prime)
    time.sleep(50)
print(dlogs)
print(primes)

n = crt(dlogs, primes)
print("prize:", n)

------ 1 ------
Elliptic Curve defined by y^2 = x^3 + 1431018*x^2 + x over Finite Field of size 57896044618658097711785492504343953926634992332820282019728792003956564819949
G.order = 10949220361 ; prime = 10949220361
------ 2 ------
Elliptic Curve defined by y^2 = x^3 + 4015385*x^2 + x over Finite Field of size 57896044618658097711785492504343953926634992332820282019728792003956564819949
G.order = 5 ; prime = 5
[+] discrete log: 2
------ 3 ------
Elliptic Curve defined by y^2 = x^3 + 2403814*x^2 + x over Finite Field of size 57896044618658097711785492504343953926634992332820282019728792003956564819949
G.order = 83 ; prime = 83
------ 4 ------
Elliptic Curve defined by y^2 = x^3 + 3132530*x^2 + x over Finite Field of size 57896044618658097711785492504343953926634992332820282019728792003956564819949
G.order = 3739139 ; prime = 3739139
------ 5 ------
Elliptic Curve defined by y^2 = x^3 + 993488*x^2 + x over Finite Field of size 57896044618658097711785492504343953926634992332820282019728